In [1]:
import pandas as pd
import os

games_info = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), "data", "raw", "games_detailed_info2025.csv"))
games_info.rename(columns={'id': 'ID'}, inplace=True)

In [2]:
bgg_reviews = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), "data", "raw", "bgg-26m-reviews.csv"))

In [3]:
import data_prep
game_features = [ 'yearpublished_bin']
data_preprocessor = data_prep.DataPreprocessor(games_info, bgg_reviews)
game_info_cut = data_preprocessor.discretize_column(games_info, 'usersrated', n_bins=10, percent=0.10, drop_original=True )
game_info_final = data_preprocessor.discretize_column(game_info_cut, 'yearpublished', n_bins=10, drop_original=True)

bgg_reviews.drop(columns=['comment', 'Unnamed: 0', 'name'], inplace=True)
data_preprocessor2 = data_prep.DataPreprocessor(game_info_final, bgg_reviews)
game_merged = data_preprocessor2.merge_datasets(featires_to_add=game_features)

In [4]:

game_merged.head()

,user,rating,ID,yearpublished_bin
0,sidehacker,10.0,13,0
1,Varthlokkur,10.0,13,0
2,dougthonus,10.0,13,0
3,cypar7,10.0,13,0
4,ssmooth,10.0,13,0


In [5]:
data_entries, num_users, num_items, num_features, user2idx, game2idx, feature2idx = data_preprocessor.data_to_tuple(game_merged, user_col='user', item_col='ID', feature_col=['yearpublished_bin'], rating_col='rating')


In [7]:
data_train, data_test = data_preprocessor.train_test_split(data_entries, test_size=0.4, seed=42)

In [ ]:
epochs = 10
k = 5
lambda_ = 0.1
tessst = TensorRecommend.TensorRecommend(k, lambda_=lambda_, eta=0.01, data_entries=data_train, num_users=num_users, 
                                             num_items=num_items, num_features=num_features, seed=42, loading=False)


for epoch in range(epochs):
    tessst.train_epoch(t=epoch+1, initial_lr=0.01, max_norm=5.0)

In [ ]:
tessst.save_model(os.path.join(os.path.dirname(os.getcwd()), "model_19/2"))